**Goal:** The aim of the challenge was to provide an opportunity for the development, testing and evaluation of AI models for automatic classification of abnormalities captured in video capsule endoscopy (VCE) video frames. It promotes the development of vendor-independent and generalized AI-based models for automatic abnormality classification pipeline with 10 class labels:
1.	Angioectasia
2.	Bleeding
3.	Erosion
4.	Erythema
5.	Foreign body
6.	Lymphangiectasia
7.	Polyp
8.	Ulcer
9.	Worms
10.	Normal

**Evaluation**
1.	Goal Metric
    •	Balanced Accuracy
    •	Mean AUC
2.	Other Metrics
    •	AUC-ROC
    •	Specificity
    •	Mean Specificity
    •	F1 Score
    •	Mean F1 Score
    •	Average Precision
    •	Mean Average Precision


**Importing necessary libraries**

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
import numpy as np
import os

**Define dataset directories**

In [2]:
train_dir = r'C:\Users\Mokshda Sharma\Desktop\My Projects\VCE_Analysis\Capsule_vision\training'
val_dir = r'C:\Users\Mokshda Sharma\Desktop\My Projects\VCE_Analysis\Capsule_vision\validation'
test_dir = r'C:\Users\Mokshda Sharma\Desktop\My Projects\VCE_Analysis\Capsule_vision\testing'

**Loading images and preprocessing them (basically resizing)**

In [3]:
# Image preprocessing and augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [4]:
# Load images from directories
train_generator = train_datagen.flow_from_directory(
    train_dir, target_size=(224, 224), batch_size=32, class_mode='categorical')

val_generator = val_datagen.flow_from_directory(
    val_dir, target_size=(224, 224), batch_size=32, class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_dir, target_size=(224, 224), batch_size=32, class_mode='categorical', shuffle=False)

Found 37607 images belonging to 10 classes.
Found 16132 images belonging to 10 classes.
Found 0 images belonging to 0 classes.


**Define CNN Model**

With the help of transfer learning, using a base model of MobileNetV2 and training a basic CNN model with that base model for better results.

In [ ]:
# base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
# base_model.trainable = False  # Freeze pre-trained layers

# model = tf.keras.Sequential([
#     base_model,
#     tf.keras.layers.GlobalAveragePooling2D(),
#     tf.keras.layers.Dense(256, activation='relu'),
#     tf.keras.layers.Dense(len(train_generator.class_indices), activation='softmax')
# ])

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [13]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l2

# Load MobileNetV2 with pre-trained weights but exclude top layers
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Freeze all layers initially
base_model.trainable = False

# Define a new model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu', kernel_regularizer=l2(0.001)),  # Added L2 regularization
    Dropout(0.4),  # Added dropout to prevent overfitting
    Dense(len(train_generator.class_indices), activation='softmax')
])

# Compile model with a small learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),  # Lower LR
              loss='categorical_crossentropy',
              metrics=['accuracy'])


# Train the model
model.fit(train_generator, 
          validation_data=val_generator, 
          epochs=5, 
          verbose=1)

Epoch 1/5
1176/1176 ━━━━━━━━━━━━━━━━━━━━ 1839s 2s/step - accuracy: 0.7433 - loss: 1.3826 - val_accuracy: 0.7855 - val_loss: 1.0336
Epoch 2/5
1176/1176 ━━━━━━━━━━━━━━━━━━━━ 664s 565ms/step - accuracy: 0.7851 - loss: 1.0198 - val_accuracy: 0.7977 - val_loss: 0.9225
Epoch 3/5
1176/1176 ━━━━━━━━━━━━━━━━━━━━ 621s 528ms/step - accuracy: 0.7967 - loss: 0.9063 - val_accuracy: 0.8068 - val_loss: 0.8498
Epoch 4/5
1176/1176 ━━━━━━━━━━━━━━━━━━━━ 617s 525ms/step - accuracy: 0.8082 - loss: 0.8381 - val_accuracy: 0.8103 - val_loss: 0.7973
Epoch 5/5
1176/1176 ━━━━━━━━━━━━━━━━━━━━ 812s 691ms/step - accuracy: 0.8117 - loss: 0.7891 - val_accuracy: 0.8125 - val_loss: 0.7645


In [14]:
# Unfreeze last 30 layers of MobileNetV2 for fine-tuning
for layer in base_model.layers[-30:]:
    layer.trainable = True

# Recompile with an even lower learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),  # Reduce LR
              loss='categorical_crossentropy',
              metrics=['accuracy'])


**Compile the model**

In [ ]:
# model.compile(optimizer='adam',
#               loss='categorical_crossentropy',
#               metrics=['accuracy'])

**Train and save the model**

In [16]:
# Train the model
# model.fit(train_generator, 
#           validation_data=val_generator, 
#           epochs=10, 
#           verbose=1)

model.fit(train_generator, 
          validation_data=val_generator, 
          epochs=10)

Epoch 1/10
1176/1176 ━━━━━━━━━━━━━━━━━━━━ 602s 512ms/step - accuracy: 0.8359 - loss: 0.6937 - val_accuracy: 0.8362 - val_loss: 0.6684
Epoch 2/10
1176/1176 ━━━━━━━━━━━━━━━━━━━━ 627s 533ms/step - accuracy: 0.8430 - loss: 0.6627 - val_accuracy: 0.8422 - val_loss: 0.6403
Epoch 3/10
1176/1176 ━━━━━━━━━━━━━━━━━━━━ 1334s 1s/step - accuracy: 0.8472 - loss: 0.6432 - val_accuracy: 0.8465 - val_loss: 0.6235
Epoch 4/10
1176/1176 ━━━━━━━━━━━━━━━━━━━━ 2287s 2s/step - accuracy: 0.8496 - loss: 0.6214 - val_accuracy: 0.8520 - val_loss: 0.6101
Epoch 5/10
1176/1176 ━━━━━━━━━━━━━━━━━━━━ 2759s 2s/step - accuracy: 0.8525 - loss: 0.6132 - val_accuracy: 0.8536 - val_loss: 0.6032
Epoch 6/10
1176/1176 ━━━━━━━━━━━━━━━━━━━━ 2259s 2s/step - accuracy: 0.8583 - loss: 0.5893 - val_accuracy: 0.8582 - val_loss: 0.5823
Epoch 7/10
1176/1176 ━━━━━━━━━━━━━━━━━━━━ 1800s 2s/step - accuracy: 0.8635 - loss: 0.5733 - val_accuracy: 0.8576 - val_loss: 0.5802
Epoch 8/10
1176/1176 ━━━━━━━━━━━━━━━━━━━━ 1880s 2s/step - accuracy: 0.86

**Model Evaluation**

In [17]:
loss, acc = model.evaluate(val_generator)
print(f"Validation Accuracy: {acc:.4f}")
print(f"Validation Loss: {loss:.4f}")

505/505 ━━━━━━━━━━━━━━━━━━━━ 436s 863ms/step - accuracy: 0.8714 - loss: 0.5327
Validation Accuracy: 0.8682
Validation Loss: 0.5428


**Using model on test data**

In [ ]:
# Function to load and preprocess test images
def load_test_images(test_dir):
    test_images = []
    image_filenames = []
    for img_name in os.listdir(test_dir):
        img_path = os.path.join(test_dir, img_name)
        img = load_img(img_path, target_size=(224, 224))  # Ensure correct size
        img_array = img_to_array(img) / 255.0  # Normalize
        test_images.append(img_array)
        image_filenames.append(img_name)
    return np.array(test_images), image_filenames

# Load test images
test_dir = r'C:\Users\Mokshda Sharma\Desktop\My Projects\VCE_Analysis\Capsule_vision\testing'
test_images, test_filenames = load_test_images(test_dir)

# Make predictions
predictions = model.predict(test_images)

# Get class labels
class_labels = list(train_generator.class_indices.keys())

# Print predictions
for i, pred in enumerate(predictions):
    predicted_class = class_labels[np.argmax(pred)]
    print(f"Image: {test_filenames[i]} -> Predicted: {predicted_class}")


**Saves the model**

In [ ]:
model.save("disease_model.h5")  # Saves the entire model (architecture + weights)